In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc
from sklearn import metrics
from lightgbm import LGBMClassifier
from collections import Counter
import datetime

C:\Users\champ\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def impute_time_zero(x):
    x = str(int(x))
    if len(x) == 1:
        x = '00000' + x
        x = datetime.datetime.strptime(x,"%H%M%S")
        return x
    if len(x) == 2:
        x = '0000' + x
        x = datetime.datetime.strptime(x,"%H%M%S")
        return x 
    if len(x) == 3:
        x = '000' + x
        x = datetime.datetime.strptime(x,"%H%M%S")
        return x 
    if len(x) == 4:
        x = '00' + x
        x = datetime.datetime.strptime(x,"%H%M%S")
        return x 
    if len(x) == 5:
        x = '0' + x
        x = datetime.datetime.strptime(x,"%H%M%S")
        return x 
    if len(x) == 6:
        x = datetime.datetime.strptime(x,"%H%M%S")
        return x
def time_to_second(x):
    x = x.time()
    hour = x.hour
    minute = x.minute
    second = x.second
    total_second = 3600*hour + 60*minute + second
    return total_second

def time_to_hour(x):
    x = x.time()
    hour = x.hour
    return hour

def cal_difference_day_transaction_frequency(x):
    sorted_locdt = sorted(x['locdt'])
    sorted_loctm = sorted(x['loctm'])
    minus_locdt = []
    minus_loctm = []
    
    for i in range(len(sorted_locdt)):
        if i == len(x)-1:
            pass
        else:
            minus_locdt.append(sorted_locdt[i+1] - sorted_locdt[i])
            
    for i in range(len(sorted_loctm)):
        if i == len(x)-1:
            pass
        else:
            minus_loctm.append(sorted_loctm[i+1] - sorted_loctm[i])
    
    if len(minus_locdt) == 0 :
        if len(minus_loctm) == 0 :
            return -1 
    
    minus_locdt = np.array(minus_locdt) * 86400
    minus_loctm = np.array(minus_loctm)
    total_second = minus_locdt + minus_loctm
    mean_frequency = np.mean(total_second)
    return mean_frequency

def cal_same_day_transaction_frequency(x):
    value_list = sorted(x)
    minus_list = []
    if len(value_list) == 1:
        return -1
    else:
        for i in range(len(x)):
            if i == len(x)-1:
                pass
            else:
                minus_list.append(value_list[i+1] - value_list[i])
        mean_frequency = np.mean(minus_list)
        return mean_frequency

In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
test_data_id = test_data['txkey']
total_data = pd.concat([train_data,test_data],sort=False)
pd.set_option('display.max_columns',500)

In [4]:
# ecfg
le = LabelEncoder()
total_data['ecfg'] = le.fit_transform(total_data['ecfg'])

#flbmk
le = LabelEncoder()
total_data['flbmk'] = le.fit_transform(total_data['flbmk'])

#flg_3dsmk
le = LabelEncoder()
total_data['flg_3dsmk'] = le.fit_transform(total_data['flg_3dsmk'])

#insfg
le = LabelEncoder()
total_data['insfg'] = le.fit_transform(total_data['insfg'])

#ovrlt
le = LabelEncoder()
total_data['ovrlt'] = le.fit_transform(total_data['ovrlt'])

#loctm
total_data['loctm'] = total_data['loctm'].apply(impute_time_zero)
total_data['hour'] = total_data['loctm'].apply(time_to_hour)
total_data['loctm'] = total_data['loctm'].apply(time_to_second)

#同一個歸戶帳戶下各信用卡交易次數 (使用中)
transaction_number = total_data.groupby('bacno')['txkey'].count().rename('same_bacno_transaction_number').reset_index()
total_data = total_data.merge(transaction_number,on='bacno',how='left')
sbctn = total_data.groupby(by=['bacno','cano'])['txkey'].count().rename('same_bacno_different_cano_trade_number').reset_index()
total_data = total_data.merge(sbctn,on=['bacno','cano'],how='left')


#同一張卡距離上一筆交易的時間差
sorted_total_data = total_data.sort_values(by=['cano','locdt','loctm'])
first_total_data = total_data.sort_values(by=['locdt','loctm']).groupby(by='cano').first().reset_index()
first_total_data['is_first'] = 1

sorted_total_data['shift_locdt'] = sorted_total_data['locdt'].shift(1).fillna(0)

sorted_total_data['shift_loctm'] = sorted_total_data['loctm'].shift(1).fillna(0)

sorted_total_data['difference_locdt'] = sorted_total_data['locdt'] - sorted_total_data['shift_locdt']
sorted_total_data['difference_loctm'] = sorted_total_data['loctm'] - sorted_total_data['shift_loctm']
sorted_total_data['total_difference_second'] = 86400 * sorted_total_data['difference_locdt'] + sorted_total_data['difference_loctm']

sorted_total_data = sorted_total_data.merge(first_total_data[['txkey','is_first']],on='txkey',how='left')
sorted_total_data['is_first'] = sorted_total_data['is_first'].fillna(0)

sorted_total_data.loc[sorted_total_data['is_first']==1,'total_difference_second'] = -1

total_data = total_data.merge(sorted_total_data[['txkey','total_difference_second']],on='txkey',how='left')

#計算同一張卡不同天的平均交易頻率
ddtf = total_data.groupby(by='cano').apply(cal_difference_day_transaction_frequency).rename('difference_day_transaction_frequency').reset_index()
total_data = total_data.merge(ddtf,on='cano',how='left')

#同一卡號購買該商品的次數
sbmstn = total_data.groupby(by=['cano','mcc'])['txkey'].count().rename('same_bacno_mcc_separate_trade_number').reset_index()
total_data = total_data.merge(sbmstn,on=['cano','mcc'],how='left')

'''
#同一張卡同一天的交易頻率(無用)
tf = total_data.groupby(by=['cano','locdt'])['loctm'].apply(cal_same_day_transaction_frequency).rename('transaction_frequency').reset_index()
total_data = total_data.merge(tf,on=['cano','locdt'],how='left')

#同一個歸戶帳號同一天使用信用卡數(無效果)
sbsducn = total_data.groupby(by=['bacno','locdt'])['cano'].unique().rename('same_bacon_same_day_use_card_number').reset_index()
sbsducn['same_bacon_same_day_use_card_number'] = sbsducn['same_bacon_same_day_use_card_number'].apply(lambda x : len(x))
total_data = total_data.merge(sbsducn,on=['bacno','locdt'],how='left')

# same bacno transaction_number(不知如何解釋)
transaction_number = total_data.groupby('bacno')['txkey'].count().rename('transaction_number').reset_index()
total_data = total_data.merge(transaction_number,on='bacno',how='left')

# 店家超額交易數量(不太合理)
soln = total_data.groupby('mchno')['ovrlt'].value_counts().rename('store_over_load_number').reset_index()
soln_y = soln.loc[soln['ovrlt']=='Y'].drop('ovrlt',axis=1)
total_data = total_data.merge(soln_y,on='mchno',how='left')
total_data['store_over_load_number'] = total_data['store_over_load_number'].fillna(0)
'''

new_train_data = total_data[:len(train_data)]
new_test_data = total_data[len(train_data):]

In [5]:
X = new_train_data.drop(['fraud_ind', 'txkey', 'locdt', 'bacno', 'cano','same_bacno_transaction_number'], axis=1)
y = new_train_data[['fraud_ind']]

test_data = new_test_data.drop(['fraud_ind', 'txkey', 'locdt', 'bacno', 'cano'], axis=1)

skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

params = {
    'task':'train',
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric':'auc',
    'learning_rate':0.01,
    'num_leaves':31,
    'scale_pos_weight':10,
}

final_result = np.zeros(len(test_data))
train_f1_score_list = []
valid_f1_score_list = []
num_boost_round = 100
for epoch, (train_index, valid_index) in enumerate(skf.split(X,y)):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_valid,y_valid)
    lightgbm = lgb.train(params,train_set=lgb_train,num_boost_round=num_boost_round,valid_sets=lgb_valid,
                         early_stopping_rounds=20,verbose_eval=False)
    
    train_pediction = lightgbm.predict(X_train)
    train_pediction = np.where(train_pediction > 0.5,1,0)
    valid_prediction = lightgbm.predict(X_valid)
    valid_prediction = np.where(valid_prediction > 0.5, 1, 0)
    
    train_f1_score = metrics.f1_score(y_train,train_pediction)
    valid_f1_score = metrics.f1_score(y_valid,valid_prediction)
    
    print('Epoch {} train f1 score : {}  valid f1 score : {}'.format(epoch,train_f1_score,valid_f1_score))
    train_f1_score_list.append(train_f1_score)
    valid_f1_score_list.append(valid_f1_score)

final_result = final_result / 10

print('train f1 cv scores:',np.mean(train_f1_score_list))
print('train f1 cv scores std:',np.std(train_f1_score_list))
print('valid f1 cv scores:',np.mean(valid_f1_score_list))
print('valid f1 cv scores std:',np.std(valid_f1_score_list))

[LightGBM] [Info] Number of positive: 18320, number of negative: 1351288
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.150854 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2634
[LightGBM] [Info] Number of data points in the train set: 1369608, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.013376 -> initscore=-4.300820
[LightGBM] [Info] Start training from score -4.300820
Epoch 0 train f1 score : 0.5876938810294869  valid f1 score : 0.5643004115226338
[LightGBM] [Info] Number of positive: 18320, number of negative: 1351288
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.178756 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2642
[LightGBM] [Info] 

In [7]:
# 0.582 0.01
feature_importance = pd.DataFrame({'feature':X.columns,'score':lightgbm.feature_importance()}).sort_values(by='score',ascending=False)
feature_importance['ratio'] = feature_importance['score'] / feature_importance['score'].sum()
feature_importance

,feature,score,ratio
19,same_bacno_different_cano_trade_number,393,0.132323
21,difference_day_transaction_frequency,388,0.130640
16,stocn,322,0.108418
23,transaction_frequency,223,0.075084
15,scity,197,0.066330
1,conam,192,0.064646
17,stscd,190,0.063973
22,same_bacno_mcc_separate_trade_number,188,0.063300
13,mchno,172,0.057912
3,csmcu,163,0.054882
